In [ ]:
#import statements, hyper params, models
import numpy as np
import sys
from sklearn.metrics import mean_squared_error
#np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.pyplot import figure
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random
from random import randrange
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import time

THRESH_HOLD=.002          #MSE value when we consider the second model to have "cracked" the first
EPOCHS = 50                #number of passes of whole data
BATCH_SIZE =  50            #size of data going through at once
LEARNING_RATE = 0.001        #how much we shift parameters during back prop
NEURONS=1024                   #number of nodes in each layer
INPUTS=1                 #number of input vars
OUTPUT=1                    #number of output
XSIZE=  100                 #size of each data set generated
BAD_LOSS_PASSES=1000000        #number of times we have small loss until we stop code
NO_TIME_INCREASES_STOP=100   #number of interations with no time increase until we stop code
TESTMODULE_LAYERS=5        #number of layers in cracking module
GROUND_TRUTH_LAYERS=5      #number of layers in ground thruth models
LINCOMBO=False               #wether to take best new time or linear combination
activation=nn.ReLU()    #activation  
TRUTHMODELSGENERATED=1     #number of perturbed models
TESTMODELSGENERATED=1
pretrained=True
DATATYPE=0                #0=data in [-1 ,1], rescaled; 1=sparse data; 2=periodic
def get_Xvals():
    if DATATYPE==0:
        return np.random.uniform( size=(XSIZE, INPUTS))*2*np.pi
    elif DATATYPE==1:
        sparsedata=(np.random.uniform( size=(XSIZE, INPUTS)))*2*np.pi
        for i in sparsedata:
            for j in i:
                if np.random.randint(0,2)==0:
                    j=0
        for i in sparsedata:
            for j in i:
                if np.random.randint(0,2)==0:
                    j=0
        return sparsedata
    else:
        return np.random.normal(size=(XSIZE, INPUTS))
        

                
    
def getdata_XY():
    X=get_Xvals()
    if DATATYPE==2:
        dummy=[(i%1)*2*np.pi for i in X]
    else:
        dummy=X
    test_data = testData(torch.FloatTensor(dummy))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist =[]
    
    
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = GroundTruth(X_batch)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return X, y_truthlist

def get_yvals(model, X_data):
    if DATATYPE==2:
        X_data=[(i%1)*2*np.pi for i in X]
    model.eval()
    test_data = testData(torch.FloatTensor(X_data))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist=[]
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    #print("MAX-MIN of yvals:", max(y_truthlist)-min(y_truthlist))
    #print(max(y_truthlist))
    #print(min(y_truthlist))
    return y_truthlist

#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

#trains a network, returns time spent training 
def trainnetwork(network, train_loader, optimizer, Time=True):
    network.to(device)
    criterion = nn.MSELoss()


    network.train()
    current_epoch=1
    last_epoch=0
    epoch_diff_counter=0
    total_counter=0
    while (epoch_diff_counter<EPOCHS and total_counter<EPOCHS*10) and current_epoch>THRESH_HOLD:
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = network(X_batch)
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()        
        
        last_epoch=current_epoch
        current_epoch=epoch_loss/len(train_loader)
        if abs(last_epoch-current_epoch)<THRESH_HOLD/10:
            epoch_diff_counter=epoch_diff_counter+1
        else:
            epoch_diff_counter=0
        total_counter=total_counter+1
            
        #print("epoch loss: ", epoch_loss/len(train_loader))
            

#returns next ground truth model
#model obtained by taking average of models with better times than previous model 
def get_new_GroundTruth_lincombo(model_list, time_list, time):
    better_model_list=[]
    for (i, model) in zip(time_list, model_list):
        if i>time:
            better_model_list.append(model)
    
    newGroundTruth=GroundTruth.get_copy()
    
    layer_weight=torch.tensor([[0]*INPUTS]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_input.weight/(len(better_model_list))
        
    layer_weight.requires_grad_()    
    newGroundTruth.layer_input.weight=torch.nn.Parameter(layer_weight)
    
    
    for i in range(GROUND_TRUTH_LAYERS):
        layer_weight=torch.tensor([[0]*NEURONS]*NEURONS, requires_grad=False)
        for bettermodel in better_model_list:
            layer_weight=layer_weight+bettermodel.layers[i].weight/(len(better_model_list))
        layer_weight.requires_grad_()
        newGroundTruth.layers[i].weight=torch.nn.Parameter(layer_weight)
        
    layer_weight=torch.tensor([0]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_output.weight/(len(better_model_list))
    layer_weight.requires_grad_()   
    newGroundTruth.layer_output.weight=torch.nn.Parameter(layer_weight)

    return newGroundTruth
        
#checks accuracy of model
def check_threshhold(network, test_loader, truth):
    total_loss=0
    network.eval()
    with torch.no_grad():
        for (X_batch, y) in zip(test_loader, truth):
            X_batch = X_batch.to(device)
            y_test_pred = network(X_batch)
            y_pred_tag=y_test_pred
            total_loss=total_loss+((y_pred_tag.cpu().numpy()-y))**2
    
    total_loss=total_loss/len(truth)

    if total_loss[0][0]<THRESH_HOLD:
        return True, total_loss[0][0]
    else:
        return False, total_loss[0][0]
                       
    
       
class arbitraryLayerModel(nn.Module):
    def __init__(self, num_layers):
        super(arbitraryLayerModel, self).__init__()
        self.layer_input=nn.Linear(INPUTS, NEURONS)
        self.layers=nn.ModuleList([nn.Linear(NEURONS, NEURONS) for i in range(num_layers)])
        
        
        self.batchnorms=nn.ModuleList([nn.BatchNorm1d(NEURONS) for i in range(num_layers)])
        self.batchnorm_input=nn.BatchNorm1d(NEURONS)
        
        self.layer_output=nn.Linear(NEURONS,OUTPUT)
    def get_copy(self):
        copied_model=arbitraryLayerModel(len(self.layers))
        copied_model.layers=nn.ModuleList()
        for i in self.layers:
            copied_model.layers.append(copy.deepcopy(i))
            
        copied_model.batchnorms=nn.ModuleList()
        for i in self.batchnorms:
            copied_model.batchnorms.append(copy.deepcopy(i))
        copied_model.layer_input=copy.deepcopy(self.layer_input)
        copied_model.layer_output=copy.deepcopy(self.layer_output)
        copied_model.batchnorm_input=copy.deepcopy(self.batchnorm_input)

        return copied_model
            
     
    def forward(self, inputs):
        #x=self.activationlayer(self.batchnorm_input(self.layer_input(inputs)))
        x=activation(self.layer_input(inputs))
        for (layer, batchnorm) in zip(self.layers, self.batchnorms):
            #x=self.activationlayer(batchnorm(layer(x)))
            x=activation(layer(x))   
                
        x=self.layer_output(x)
        
        return(x)

    
    
    def change_node_model(self):
        changed_node=self.get_copy()
        NodeNum=randrange(NEURONS*(GROUND_TRUTH_LAYERS))
        for i in range(NodeNum):
            layeri=randrange(GROUND_TRUTH_LAYERS+2)
            if layeri==0:
                nodei=[randrange(INPUTS), randrange(NEURONS)]
                layeri=1
            if layeri==GROUND_TRUTH_LAYERS+1:
                 nodei=[randrange(NEURONS), randrange(OUTPUT)]
            else:
                nodei=[randrange(NEURONS), randrange(NEURONS)]
            node=[layeri, nodei]

            weightsign=randrange(2)
            if weightsign==0:
                weightsign=-1
                
            weightchange=np.random.normal(weightsign*2,1)      

            if node[0]==0:
                zeros=[[0]*INPUTS]*NEURONS
                zeros[node[1][1]][node[1][0]]=weightchange
                newweight=self.layer_input.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_input.weight=torch.nn.Parameter(newweight)    
            elif node[0]==GROUND_TRUTH_LAYERS+1:
                zeros=[[0]*(NEURONS)]*OUTPUT
                zeros[node[1][1]][node[1][0]]=weightchange
                newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
                newweight.requires_grad_()
                changed_node.layer_output.weight=torch.nn.Parameter(newweight) 

            else:
                zeros = [[0.0]*NEURONS]*NEURONS
                for layer, i in enumerate(changed_node.layers):
                    if i==node[0]+1:
                        zeros[node[1][0]][node[1][1]]=weightchange
                        newweight=layer.weight.requires_grad_(False)+torch.tensor(weightchange,requires_grad=False)
                        newweight.requires_grad_()
                        layer.weight=torch.nn.Parameter(newweight)



        return changed_node

def sine(data):
    y=[]
    for inputs in data:
        answers=[]
        for i in range(1, OUTPUT+1):
            answers.append(np.sin(sum(inputs)*i))
            
        y.append(answers)
    return y
            
        
    
    
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   





#only 1 ground truth generated
Iteration_time_list=[]   
current_time_list=[]
data_list=[]

GroundTruth = arbitraryLayerModel(GROUND_TRUTH_LAYERS)

#pretrain first ground truth
if pretrained==True:
    ground_truth_optim = optim.Adam(GroundTruth.parameters(), lr=LEARNING_RATE)
    X=np.random.uniform( size=(10000, INPUTS))*2*np.pi
    y=np.sin(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)



    GroundTruth.to(device)
    criterion = nn.MSELoss()
    GroundTruth.train()
    loss=100
    while(loss>.05):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            ground_truth_optim.zero_grad()
            y_pred = GroundTruth(X_batch)
            #print(y_pred)
            #print(y_batch)
            loss = criterion(y_pred, target=y_batch)
            loss.backward()
            ground_truth_optim.step()
            epoch_loss += loss.item()        

        print("epoch loss for pretraining: ", epoch_loss/len(train_loader))
        loss=epoch_loss/len(train_loader)


possible_infinite_time=arbitraryLayerModel(GROUND_TRUTH_LAYERS)
firstmodeltrained=arbitraryLayerModel(TESTMODULE_LAYERS)
test_model_optimizer = optim.SGD(firstmodeltrained.parameters(), lr=LEARNING_RATE)
accepted=False



while((not accepted)):
    X, y=getdata_XY()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    trainnetwork(firstmodeltrained, train_loader,test_model_optimizer)
    accepted, dummy=check_threshhold(firstmodeltrained, test_loader, y_test)
    #print(dummy)



current_time=len(data_list)
print("First time: ", current_time)
current_time_list.append(current_time)
Iteration_time_list.append(current_time)

#iterates
interationcounter=1
no_increasecounter=0
time_increase=True
loss_difference_counter=0
stop_itterations=False

while(time_increase):
    time_list=[]
    model_list=[]
    test_model_list=[arbitraryLayerModel(TESTMODULE_LAYERS) for i in range(TESTMODELSGENERATED)]

    
    
    for i in range(TRUTHMODELSGENERATED):
        data_test_list=[]
        infinite_time_counter=0
        prospective_GroundTruth=GroundTruth.change_node_model()
        model_list.append(prospective_GroundTruth)
        
        for j, test_model in enumerate(test_model_list):
            current_loss=0
            previous_loss=0
            data_counter=0
            loss_difference_counter=0

            
            train_testmodel=test_model.get_copy()
            test_model_optimizer = optim.SGD(train_testmodel.parameters(), lr=LEARNING_RATE)
            inital_data_length=len(data_list)
            
            accepted=False
            Model_beaten=False

            while(not accepted): 
                #use previously generated data
                if data_counter<inital_data_length:
                    X=data_list[data_counter]
                    y=get_yvals(prospective_GroundTruth, X)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)



                    trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    previous_loss=current_loss
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=abs(previous_loss-current_loss)
                    data_counter=data_counter+1
                    test_time=data_counter
                #generate new data if needed
                else:                          
                    X=get_Xvals()
                    data_list.append(X)
                    y=get_yvals(prospective_GroundTruth, X)
                    #if len(data_list)%20==0:
                    #    print(max(y)-min(y))


                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1)

                    previous_loss=current_loss
                    trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=abs(previous_loss-current_loss)
                    test_time=len(data_list)


                #checks if loss difference is small
                if abs(loss_diff)<(THRESH_HOLD/BAD_LOSS_PASSES):
                    loss_difference_counter=loss_difference_counter+1
                    if loss_difference_counter>BAD_LOSS_PASSES:
                        infinite_time_counter=infinite_time_counter+1
                        print("Test model beaten. Total beaten by prospective ground truth: ", infinite_time_counter)
                        break

            time_list.append(test_time)

    
    max_new_time=sum(time_list)/TESTMODELSGENERATED
    Iteration_time_list.append(max_new_time)      

    #L2 distance from$"0"$
    distance=0
    distance=distance+prospective_GroundTruth.layer_input.weight.mul(prospective_GroundTruth.layer_input.weight).sum().item()
    for layer in prospective_GroundTruth.layers:
        distance=distance+layer.weight.mul(layer.weight).sum().item()
    distance=distance+prospective_GroundTruth.layer_output.weight.mul(prospective_GroundTruth.layer_output.weight).sum().item()
    distance=distance/(NEURONS*GROUND_TRUTH_LAYERS+INPUTS*NEURONS+OUTPUT*NEURONS)
    
    #singular values
    product=torch.eye(NEURONS)
    for layer in prospective_GroundTruth.layers:
        product=product@layer.weight
    numpproduct=product.detach().numpy()
    u, s, vh = np.linalg.svd(numpproduct, full_matrices=True)
    singular=s
    
    
    #prints iteration data
    print("Iteration", interationcounter,": | Data sizes: ", time_list)   
    print(f'Iteration {interationcounter}: | Distance: {distance: .3f}')
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f} | # beaten: {infinite_time_counter}')
    interationcounter=interationcounter+1
    
    if infinite_time_counter==10:
        print("Potential unbreakable found")
        possible_infinite_time=prospective_GroundTruth.get_copy()
        print("Time: ", time_list)
        break
    
     
    
    


    
    
    #changes ground truth if better time found
    if max_new_time>current_time:
        GroundTruth=model_list[0].get_copy()
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
    else:
        no_increasecounter=no_increasecounter+1

    current_time_list.append(current_time)
        
    #if no better ground truth found for specified number, stops code                 
    if no_increasecounter==NO_TIME_INCREASES_STOP:
        print("No increase for ", NO_TIME_INCREASES_STOP,  " iterations")
        time_increase=False







 

epoch loss for pretraining:  0.06975987471996632
epoch loss for pretraining:  0.006088731123782054
0.34576052
0.2398264
0.1670857
0.14624049
0.12048188
0.1347118
0.12698027
0.15311857
0.10369847
0.10565291
0.13205941
0.119047746
0.110962115
